# Calculating image metrics

In [1]:
import sys
sys.path.insert(0, '/datadrive/facediffusion')
from fdh256_dataset import FDF256Dataset

Could not load pyspng. Defaulting to pillow image backend.


In [7]:
dataset_path = '/datadrive/FDF/dataset/val'
dataset = FDF256Dataset(dirpath=dataset_path, load_keypoints=True, load_impath=True)

Dataset loaded from: /datadrive/FDF/dataset/val. Number of samples:6531


In [3]:
EMA = False # set to true to perform evaluation on the EMA model's generated images

In [4]:
if EMA:
    generated_ims_path = '/datadrive/facediffusion/Coginfocom/images-ema'
    sr_ims_path = '/datadrive/facediffusion/Coginfocom/DFDNet-SR-Coginfocom-EMA'
else:
    generated_ims_path = '/datadrive/facediffusion/Coginfocom/images'
    sr_ims_path = '/datadrive/facediffusion/Coginfocom/DFDNet-SR-Coginfocom'

In [5]:
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
import tensorflow as tf
import torch
import os
import PIL
from PIL import Image
import numpy as np
import logging

logging.basicConfig(filename="EMA-DFDNet" if EMA else "non-EMA-DFDNet",
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)
logging.info("Running image metrics comparison...")
logger = logging.getLogger('DFDNet')

2023-05-10 08:35:28.108955: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 08:35:28.969898: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
total_pred_psnr = 0.0
total_pred_ssim = 0.0
total_sr_psnr = 0.0
total_sr_ssim = 0.0

for i in tqdm(range(len(dataset))):
    original_img = dataset[i]["img"]
    img_name: str = str(dataset[i]['impath']).split('/')[-1]
    img_idx = img_name.split('.')[0]
    
    pred_img_path = os.path.join(generated_ims_path, f'{img_idx}.jpeg')
    pred_img = Image.open(pred_img_path)
    pred_img = pred_img.resize((256, 256), Image.BICUBIC)
    pred_img = np.asarray(pred_img)
    
    sr_img_path = os.path.join(sr_ims_path, f'{img_idx}.png')
    sr_img = Image.open(sr_img_path)
    sr_img = np.asarray(sr_img)
    
    pred_im_psnr: float = tf.image.psnr(pred_img, original_img, max_val=255)
    logger.info(f'{img_idx} PRED PSNR {pred_im_psnr}')
    pred_im_ssim: float = ssim(im1=original_img, im2=pred_img, data_range=pred_img.max() - pred_img.min(), channel_axis=2)
    logger.info(f'{img_idx} PRED SSIM {pred_im_ssim}')
    total_pred_psnr += pred_im_psnr
    total_pred_ssim += pred_im_ssim
    
    sr_im_psnr: float = tf.image.psnr(sr_img, original_img, max_val=255)
    logger.info(f'{img_idx} SR PSNR {sr_im_psnr}')
    sr_im_ssim: float = ssim(im1=original_img, im2=sr_img, data_range=sr_img.max() - sr_img.min(), channel_axis=2)
    logger.info(f'{img_idx} SR SSIM {sr_im_ssim}')
    total_sr_psnr += sr_im_psnr
    total_sr_ssim += sr_im_ssim
    
avg_pred_psnr = total_pred_psnr / len(dataset)
avg_pred_ssim = total_pred_ssim / len(dataset)

avg_sr_psnr = total_sr_psnr / len(dataset)
avg_sr_ssim = total_sr_ssim / len(dataset)

logger.info(f'Interpolated images PSNR = {avg_pred_psnr}')
logger.info(f'SR images PSNR = {avg_sr_psnr}')
logger.info(f'Interpolated images SSIM = {avg_pred_ssim}')
logger.info(f'SR images SSIM = {avg_sr_ssim}')

print(f'Interpolated images PSNR = {avg_pred_psnr}')
print(f'SR images PSNR = {avg_sr_psnr}')
print(f'Interpolated images SSIM = {avg_pred_ssim}')
print(f'SR images SSIM = {avg_sr_ssim}')
    

  0%|          | 0/6531 [00:00<?, ?it/s]2023-05-10 08:36:51.570780: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14087 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0001:00:00.0, compute capability: 7.0
2023-05-10 08:36:51.580393: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14087 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0002:00:00.0, compute capability: 7.0
2023-05-10 08:36:51.581176: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14087 MB memory:  -> device: 2, name: Tesla V100-PCIE-16GB, pci bus id: 0003:00:00.0, compute capability: 7.0
2023-05-10 08:36:51.581708: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 14087 MB memory:  -> device: 3

Interpolated images PSNR = 21.267093658447266
SR images PSNR = 19.325897216796875
Interpolated images SSIM = 0.6628624857274609
SR images SSIM = 0.5971902770380556
